In [ ]:
from models.data_models import Document
from openai_api import *

import json
from tqdm import tqdm

from openai import AzureOpenAI
import azure.keyvault.secrets as azk
from azure.identity import DefaultAzureCredential

from sklearn.model_selection import train_test_split
import spacy
from spacy.tokens import DocBin


In [ ]:
'''
def clean(text):
    text = text.strip().replace("\n", " ").replace("\r", "").replace("\t", " ")
    text = re.sub('\\s+', ' ', text)
    return text


resumes = []
for file in tqdm(os.listdir(f'{os.getcwd()}/DATA/data_en')):
    with open(f'{os.getcwd()}/DATA/data_en/{file}','r') as f:
        contents = clean(f.read())
        resumes.append(contents)

with open('resumes.json','x') as j:
    j.write(json.dumps(resumes,indent=4,ensure_ascii=False))
'''

In [ ]:
'''
with open('resumes.json','r') as f:
    resumes = json.loads(f.read())

    
labeled_resumes = []

for i in tqdm(range(len(resumes[:500]))):
    text = resumes[i]
    api_labels = await send_request(text)

    labels = []
    for l in api_labels or []:
        label = l['type']
        content = l['text']
        start = text.find(content)
        if start == -1:
            continue
        end = start + len(content)
        labels.append(Label(start,end,label,content).to_dict())

    doc = Document(i,text,labels).to_dict()
    labeled_resumes.append(doc)
'''

100%|██████████| 500/500 [34:43<00:00,  4.17s/it]  


In [13]:
with open('cleaned_resumes.json','r') as j:
    labeled_resumes = json.loads(j.read())

In [9]:
cleaned_resumes = []

delete = ['appraisal','view','account','events','written','verbal','network','application','concepts','develop','copy','mechanical','structural','safety','conflict','culture','organizational','phone','taxes','automation','profit','skills','pc skills','research','practices','procedures','experience','skill','retail','B.S.','Healthcare','Network','policies','financial','automotive','oral','interpersonal','website','workbench','video','book','medallion','budget','agency','automate','client','clients','plan','meetings','proposals','strategy']
deletewords = ['company name','community college','20','college','major']

for r in labeled_resumes:
  name, text, labels = r['id'],r['content'], r['labels']
  clean_labels = []
  starts = []

  for l in labels:
    start, end, label, value = l["start"], l["end"], l["label"], l["value"]
    
    if (start in starts) | (value.lower() in delete) | (len(value) >= 40):
        continue

    starts.append(start)
    clean_labels.append(l)

  cleaned_resumes.append(Document(name,text,clean_labels).to_dict())

In [9]:
with open('cleaned_resumes.json','w') as j:
  j.write(json.dumps(cleaned_resumes))

In [14]:
train, test = train_test_split(cleaned_resumes, test_size=0.2, random_state=777)

In [15]:
nlp = spacy.blank('en')
doc_bin = DocBin()


In [16]:
def has_overlap(span1, span2):
    return not (span1.end <= span2.start or span2.end <= span1.start)

empty_count, ent_count, overlap_count = 0, 0, 0

for r in train:
    debug_id, text, labels = r["id"],r["content"],r["labels"]
    doc = nlp(text)
    ents = []
    for l in labels:
        start, end, label, value = l["start"], l["end"], l["label"], l["value"]
        span = doc.char_span(start_idx=start, end_idx=end, label=label)
        if span is None:
            empty_count +=1
            continue
        if any(has_overlap(span, existing_span) for existing_span in ents):
            overlap_count +=1
            continue
        
        ents.append(span)
    ent_count += len(ents)
    doc.ents = ents 
    doc_bin.add(doc) 
print(empty_count, ent_count, overlap_count)
doc_bin.to_disk("train.spacy")

425 27698 411


In [18]:
empty_count, ent_count, overlap_count = 0, 0, 0

for r in test:
    debug_id, text, labels = r["id"],r["content"],r["labels"]
    doc = nlp(text)
    ents = []
    for l in labels:
        start, end, label, value = l["start"], l["end"], l["label"], l["value"]
        span = doc.char_span(start, end, label=label)
        if span is None:
            empty_count += 1
            continue
        if any(has_overlap(span, existing_span) for existing_span in ents):
            overlap_count += 1
            continue
        
        ents.append(span)
    ent_count += len(ents)
    doc.ents = ents 
    doc_bin.add(doc) 
print(empty_count, ent_count)
doc_bin.to_disk("test.spacy")

118 7932


In [20]:
! python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency  --force

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [21]:
!python -m spacy debug data config.cfg --paths.train ./train.spacy --paths.dev ./test.spacy


============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: en
Training pipeline: tok2vec, ner
1964 training docs
2455 evaluation docs
⚠ 1953 training examples also in evaluation data
⚠ Low number of examples to train a new pipeline (1964)

============================== Vocab & Vectors ==============================
ℹ 1866208 total word(s) in the data (52346 unique)
ℹ No word vectors present in the package

========================== Named Entity Recognition ==========================
ℹ 2 label(s)
0 missing value(s) (tokens with '-' label)
✔ Good amount of examples for all labels
✔ Examples without occurrences available for all labels
✔ No entities consisting of or starting/ending with whitespace
✔ No entities crossing sentence boundaries

================================== Summary ==================================


In [9]:
!python -m spacy train config.cfg --output ./ --paths.train ./train.spacy --paths.dev ./test.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    375.90    0.40    0.25    0.95    0.00
  0     200       2090.17  12718.38    1.08   25.45    0.55    0.01
  0     400        156.17   5252.85    1.64   32.02    0.84    0.02
  0     600        185.26   5615.28    0.01   28.57    0.01    0.00
  0     800        184.45   5702.18    0.01   40.00    0.01    0.00
  0    1000        203.40   5350.71    0.66   40.00    0.33    0.01
  0    1200        281.49   5122.71    1.56   31.02    0.80    0.02
  0    1400       4257.51   5936.71    4.79   40.53    2.55    0.05
  0    1600        307.22   5240.45 